In [ ]:
# !pip install scikit-learn==1.5.2 
# !pip install autogluon.tabular
# !pip install ray==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 72.6 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 6.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import xgboost 
print(xgboost .__version__)


2.0.3


In [3]:
import os
import gc
import json
import pickle
import shutil
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression, Ridge

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier, early_stopping
from xgboost import XGBClassifier

from autogluon.tabular import TabularPredictor

from scipy.special import logit

import optuna


import warnings
warnings.filterwarnings('ignore')

In [4]:
class CFG:
    train_path = '/kaggle/input/playground-series-s4e11/train.csv'
    test_path = '/kaggle/input/playground-series-s4e11/test.csv'
    sample_sub_path = '/kaggle/input/playground-series-s4e11/sample_submission.csv'
    
    original_data_path = '/kaggle/input/depression-surveydataset-for-analysis/final_depression_dataset_1.csv'
    
    target = 'Depression'
    n_folds = 5
    seed = 42

In [5]:
def process_detailed_impute(df_train, df_test, df_original, model_name='lgbm'):
    """
    Áp dụng tất cả các phép gán, map, và imputation chi tiết.
    ĐÃ SỬA: Chấp nhận model_name và xử lý lỗi CatBoost.
    """
    train = df_train.copy()
    test = df_test.copy()
    original = df_original.copy()
    all_dfs = [train, test, original] 

    # --- Định nghĩa các ánh xạ (maps) ---
    gender_map = {'Male': 1, 'Female': 0}
    work_map = {'Working Professional': 1, 'Student': 0}
    suicidal_map = {'Yes': 1, 'No': 0}
    family_map = {'Yes': 1, 'No': 0}
    sleep_map = {
        'Less than 5 hours': 4, '7-8 hours': 7.5, 'More than 8 hours': 9,
        '5-6 hours': 5.5, '3-4 hours': 3.5,
    }
    diet_map = {'Healthy': 2, 'Moderate': 1, 'Unhealthy': 0}

    for df in all_dfs:
        try:
            # --- Áp dụng Ánh xạ (ĐÃ SỬA LỖI TÊN BIẾN) ---
            df['Gender'] = df['Gender'].map(gender_map)
            df['Working Professional or Student'] = df['Working Professional or Student'].map(work_map) 
            df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].map(suicidal_map) 
            df['Family History of Mental Illness'] = df['Family History of Mental Illness'].map(family_map) 
            df['Sleep Duration'] = df['Sleep Duration'].map(sleep_map)
            df['Dietary Habits'] = df['Dietary Habits'].map(diet_map) 

            # --- Xử lý Imputation có điều kiện (ĐÃ SỬA LỖI TÊN CỘT) ---
            for col in ['Work Pressure', 'Job Satisfaction']:
                df[col] = df.apply(
                    lambda row: 0 if row['Working Professional or Student'] == 0 and pd.isnull(row[col]) else row[col],
                    axis=1
                )
            for col in ['Academic Pressure', 'Study Satisfaction']:
                df[col] = df.apply(
                    lambda row: 0 if row['Working Professional or Student'] == 1 and pd.isnull(row[col]) else row[col],
                    axis=1
                )
        except KeyError as e:
            print(f"Cảnh báo: Không tìm thấy cột {e} trong process_detailed_impute. Bỏ qua...")
            pass

    
    # --- Xử lý Imputation còn lại ---
    numeric_cols = ['Age', 'CGPA', 'Work/Study Hours']
    categorical_cols = ['Academic Pressure', 'Work Pressure', 'Study Satisfaction', 
                    'Job Satisfaction', 'Financial Stress', 'Sleep Duration', 'Dietary Habits',
                    'Name', 'City', 'Profession', 'Degree']
    for col in numeric_cols:
        if col in train.columns:
            med = train[col].median()
            for df in all_dfs:
                df[col] = df[col].fillna(med)

    for col in categorical_cols:
        if col in train.columns:
            mode_val = train[col].mode()[0]
            for df in all_dfs:
                df[col] = df[col].fillna(mode_val)
    
    if model_name == 'cb':
        cat_cols = train.columns.tolist() 
    else:
        cat_cols = train.select_dtypes(include='object').columns
    
    for df in [train, test, original]:
        for col in cat_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).astype('category')

    if model_name != 'cb':
        for col in cat_cols:
            if all(col in df.columns for df in [train, test, original]):
                try:
                    common_cats = train[col].cat.categories.union(original[col].cat.categories)
                    train[col] = train[col].cat.set_categories(common_cats)
                    original[col] = original[col].cat.set_categories(common_cats)
                    test[col] = test[col].cat.set_categories(common_cats)
                except Exception:
                    pass

            
    return train, test, original

In [6]:
def process_native(df_train, df_test, df_original, model_name='lgbm'):
    """
    PHIÊN BẢN ĐÃ SỬA LỖI
    Chuyển các cột 'object' sang 'category'.
    Áp dụng chiến lược đặc biệt (tất cả là category) cho CatBoost.
    """
    train_df = df_train.copy()
    test_df = df_test.copy()
    original_df = df_original.copy()
 
    if model_name == 'cb':
        cat_cols = train_df.columns.tolist() 
    else:
        cat_cols = train_df.select_dtypes(include='object').columns
    
    for df in [train_df, test_df, original_df]:
        for col in cat_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).astype('category')
            
    # Đồng bộ categories
    for col in cat_cols:
        if col in original_df.columns and col in train_df.columns and col in test_df.columns:
            try:
                common_cats = train_df[col].cat.categories.union(original_df[col].cat.categories)
                train_df[col] = train_df[col].cat.set_categories(common_cats)
                original_df[col] = original_df[col].cat.set_categories(common_cats)
                test_df[col] = test_df[col].cat.set_categories(common_cats)
            except Exception:
                pass 
            
    return train_df, test_df, original_df

In [7]:
def get_data(pipeline_type='p1', pseudo_data=None, model_name='lgbm'):
    # --- 1. Load dữ liệu ---
    df_train = pd.read_csv(CFG.train_path)
    df_test = pd.read_csv(CFG.test_path)
    df_original = pd.read_csv(CFG.original_data_path)
    
    for df in [df_train, df_test]:
        if 'id' in df.columns: df.drop('id', axis=1, inplace=True)
    
    # Map target nếu là object
    for df in [df_original]:
        if df[CFG.target].dtype == 'object':
            df[CFG.target] = df[CFG.target].map({'Yes':1, 'No':0})
    
    y_train = df_train[CFG.target]
    X_train = df_train.drop(CFG.target, axis=1)
    y_original = df_original[CFG.target]
    X_original = df_original.drop(CFG.target, axis=1)
    X_test = df_test.copy()
    
    # --- 2. Xử lý dữ liệu ---
    if pipeline_type in ['p1', 'p3']:
        X_train_proc, X_test_proc, X_orig_proc = process_native(X_train, X_test, X_original, model_name=model_name)
    elif pipeline_type in ['p2', 'p4']:
        X_train_proc, X_test_proc, X_orig_proc = process_detailed_impute(X_train, X_test, X_original, model_name=model_name)
    else:
        raise ValueError("pipeline_type không hợp lệ")
    
    # --- 3. Kết hợp dữ liệu ---
    common_cols = [c for c in X_train_proc.columns if c in X_orig_proc.columns]

    if pipeline_type in ['p1', 'p2']:
        X_final = pd.concat([X_train_proc[common_cols], X_orig_proc[common_cols]], ignore_index=True)
        y_final = pd.concat([y_train, y_original], ignore_index=True)
    elif pipeline_type in ['p3', 'p4']:
        if pseudo_data is None:
            raise ValueError("Cần pseudo_data cho pipeline p3 hoặc p4")
        X_pseudo, y_pseudo = pseudo_data
        X_final = pd.concat([X_train_proc[common_cols], X_orig_proc[common_cols], X_pseudo[common_cols]], ignore_index=True)
        y_final = pd.concat([y_train, y_original, y_pseudo], ignore_index=True)
    
    # --- 4. Xử lý X_test ---
    if pipeline_type in ['p1', 'p3']:
        X_test_final = X_test_proc
    else:  # p2, p4
        X_test_final = X_test_proc[common_cols]
    
    return X_final, y_final, X_test_final


In [ ]:
class Trainer:
    def __init__(self, model, config=CFG):
        self.model = model
        self.config = config

    def fit_predict(self, X, y, X_test, threshold=0.5):
        # --- Convert X, y, X_test thành DataFrame/Series ngoài loop ---
        if not isinstance(X, pd.DataFrame): X = pd.DataFrame(X)
        if not isinstance(y, pd.Series): y = pd.Series(y)
        if not isinstance(X_test, pd.DataFrame): X_test = pd.DataFrame(X_test)

        print(f'Training {self.model.__class__.__name__}\n')

        scores = []    
        oof_pred_probs = np.zeros(X.shape[0])
        test_pred_probs = np.zeros(X_test.shape[0])
        
        skf = StratifiedKFold(
            n_splits=self.config.n_folds, 
            random_state=self.config.seed, 
            shuffle=True
        )
        
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            # --- Tạo bản copy cho fold hiện tại ---
            X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            X_test_fold = X_test.copy() 

            model = clone(self.model)

            # --- CatBoost ---
            if isinstance(model, CatBoostClassifier):
                cat_features = X_train.columns.tolist()
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_val, y_val)],
                    cat_features=cat_features,
                    early_stopping_rounds=100,
                    verbose=0
                )

            # --- LGBM ---
            elif isinstance(model, LGBMClassifier):
                cat_features = X_train.select_dtypes(include='category').columns.tolist()
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_val, y_val)],
                    eval_metric='logloss', 
                    categorical_feature=cat_features,
                    callbacks=[early_stopping(100, verbose=False)]
                )

            # --- XGB ---
            elif isinstance(model, XGBClassifier):
                # Chuyển các cột category sang int riêng cho fold
                cat_cols = X_train.select_dtypes(include='category').columns
                for col in cat_cols:
                    X_train[col] = X_train[col].cat.codes
                    X_val[col] = X_val[col].cat.codes
                    X_test_fold[col] = X_test_fold[col].cat.codes
                
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_val, y_val)],
                    eval_metric='logloss',
                    early_stopping_rounds=100,
                    verbose=False
                )

            else:
                model.fit(X_train, y_train)
            
            # --- Dự đoán OOF và Test ---
            y_pred_probs = model.predict_proba(X_val)[:, 1]
            oof_pred_probs[val_idx] = y_pred_probs  

            test_pred_probs += model.predict_proba(X_test_fold)[:, 1] / self.config.n_folds
            
            score = accuracy_score(y_val, (y_pred_probs > threshold).astype(int))
            scores.append(score)
            print(f'--- Fold {fold_idx + 1} - Accuracy: {score:.6f}')
            
            del model, X_train, y_train, X_val, y_val, y_pred_probs, X_test_fold
            gc.collect()
            
        # --- Overall Score ---
        overall_score = accuracy_score(y, (oof_pred_probs > threshold).astype(int))
        print(f'\n------ Overall: {overall_score:.6f} | Average: {np.mean(scores):.6f} ± {np.std(scores):.6f}')
        
        return oof_pred_probs, test_pred_probs, scores

In [9]:
# --- CÁC HÀM HELPER KHÁC ---
def find_best_threshold(y_true, oof_probs):
    if oof_probs.ndim == 2:
        oof_probs = oof_probs[:, 1]
    best_threshold = 0.5
    best_score = 0
    for threshold in np.arange(0.0, 1.01, 0.01):
        preds = (oof_probs > threshold).astype(int)
        score = accuracy_score(y_true, preds)
        if score > best_score:
            best_score = score
            best_threshold = threshold
    print(f"Tìm thấy Ngưỡng Tốt Nhất: {best_threshold:.2f} | OOF Accuracy: {best_score:.6f}")
    return best_threshold, best_score

def save_submission(name, test_pred_probs, score, threshold=0.5):
    os.makedirs('submissions', exist_ok=True)
    sub = pd.read_csv(CFG.sample_sub_path)
    sub[CFG.target] = (test_pred_probs > threshold).astype(int)
    file_path = f'submissions/sub_{name}_{score:.6f}.csv'
    sub.to_csv(file_path, index=False)
    print(f"Submission đã lưu tại: {file_path}")

In [ ]:
def train_autogluon_model(X, y, X_test, model_name, time_limit_sec=3600):
    print(f'Training AutoGluon model: {model_name}\n')
    model_path = f'./autogluon_base_models/{model_name}'
    if os.path.exists(model_path):
        shutil.rmtree(model_path)
    
    train_data = X.copy()
    train_data[CFG.target] = y
    
    predictor = TabularPredictor(
        label=CFG.target,
        eval_metric='accuracy',
        path=f'./autogluon_base_models/{model_name}'
    )
    
    predictor.fit(
        train_data,
        time_limit=time_limit_sec,
        presets='high_quality',
        num_bag_folds=CFG.n_folds,
        num_bag_sets=1,         
        num_stack_levels=0,     
        fit_weighted_ensemble=True,
        verbosity=1
    )
    
    # OOF
    oof_preds_df = predictor.predict_proba_oof() 
    oof_preds_series_1d = oof_preds_df[predictor.class_labels[1]]
    oof_preds_full = oof_preds_series_1d.reindex(X.index).values 
    
    # Test
    test_preds_df = predictor.predict_proba(X_test)
    test_preds = test_preds_df[predictor.class_labels[1]].values 
    
    # Score
    lb = predictor.leaderboard(silent=True)
    overall_score = lb['score_val'].iloc[0]
    
    print(f'\n------ AutoGluon Overall Score: {overall_score:.6f}')
    
    return oof_preds_full, test_preds, [overall_score]

In [ ]:
lr_params = {
    "C": 5.559884346567435,
    "max_iter": 1000,
    "n_jobs": 4,
    "penalty": "l2",
    "random_state": 42,
    "solver": "newton-cg",
    "tol": 0.08313081991676836
}

cb_params = {
    "task_type": "GPU",
    "border_count": 180,
    "depth": 4,
    "iterations": 2372,
    "l2_leaf_reg": 4.442847441200204,
    "learning_rate": 0.0514109059943355,
    "min_child_samples": 146,
    "random_state": 42,
    "random_strength": 0.18678416655567043,
    "scale_pos_weight": 1.019889465491297,
    "subsample": 0.3511896501762123,
    "verbose": False,
    "bootstrap_type": "Bernoulli"
}

xgb_params = {
    "tree_method": "gpu_hist",
    "colsample_bylevel": 0.25155109886677396,
    "colsample_bynode": 0.5723191165109757,
    "colsample_bytree": 0.18034301813835885,
    "enable_categorical": True,
    "gamma": 3.6392698070258622,
    "max_bins": 26161,
    "max_depth": 16,
    "max_leaves": 67,
    "min_child_weight": 34,
    "n_estimators": 3853,
    "n_jobs": -1,
    "random_state": 42,
    "reg_alpha": 7.996080341061729,
    "reg_lambda": 46.83054555763492,
    "scale_pos_weight": 1.2157646356820928,
    "subsample": 0.9117754083869292,
    "verbosity": 0
}

lgbm_params = {
    "device": "gpu", 
    "boosting_type": "gbdt",
    "colsample_bytree": 0.18283018243382332,
    "learning_rate": 0.09945326391012832,
    "max_bins": 36644,
    "min_child_samples": 105,
    "min_child_weight": 0.2083765599710974,
    "n_estimators": 244,
    "n_jobs": -1,
    "num_leaves": 122,
    "random_state": 42,
    "reg_alpha": 8.662578235164972,
    "reg_lambda": 3.5696291074963926,
    "scale_pos_weight": 1.0733293968870794,
    "subsample": 0.5360642841695424,
    "verbose": -1
}

lgbm_goss_params = {
    "device": "gpu",
    "boosting_type": "goss",
    "colsample_bytree": 0.11905309670044416,
    "learning_rate": 0.04641567005485582,
    "max_bins": 18501,
    "min_child_samples": 283,
    "min_child_weight": 0.5242575557671028,
    "n_estimators": 966,
    "n_jobs": -1,
    "num_leaves": 159,
    "random_state": 42,
    "reg_alpha": 8.160196501794122,
    "reg_lambda": 9.861767101758469,
    "scale_pos_weight": 0.991416082619142,
    "subsample": 0.9259030065865966,
    "verbose": -1
}

lgbm_dart_params = {
    "device": "gpu",
    "boosting_type": "dart",
    "colsample_bytree": 0.1181378019860333,
    "learning_rate": 0.0910272033595692,
    "max_bins": 7386,
    "min_child_samples": 225,
    "min_child_weight": 0.28492487885169293,
    "n_estimators": 1454,
    "n_jobs": -1,
    "num_leaves": 163,
    "random_state": 42,
    "reg_alpha": 2.9686583338116925,
    "reg_lambda": 9.781841345026509,
    "scale_pos_weight": 0.8309734952725212,
    "subsample": 0.511061530253864,
    "verbose": -1
}

lr_model_pipeline = make_pipeline(
    OneHotEncoder(handle_unknown='ignore', min_frequency=0.001),
    LogisticRegression(**lr_params)
)

model_definitions = {
    'xgb': XGBClassifier(**xgb_params),
    'lgbm': LGBMClassifier(**lgbm_params),
    'cb': CatBoostClassifier(**cb_params),
    'lr': lr_model_pipeline,
    'goss': LGBMClassifier(**lgbm_goss_params),
    'dart': LGBMClassifier(**lgbm_dart_params)
}

pipeline_list = ['p1', 'p2']

In [14]:
all_oof_preds = {}
all_test_preds = {}
all_scores = {} 
model_best_oof_scores = {}
y_true=None

In [ ]:
for model_name, model_obj in model_definitions.items():
    for pipeline_name in pipeline_list:
        if model_name == 'cb' and pipeline_name in ['p1']:
            continue
            
        print(f"\n--- PROCESSING: {model_name.upper()} on {pipeline_name.upper()} ---")
        X, y, X_test = get_data(
            pipeline_type=pipeline_name, 
            model_name=model_name 
        )
        if y_true is None: y_true = y.copy()
        
        full_model_name = f"{model_name}_{pipeline_name}"
        trainer = Trainer(model_obj)
        oof_preds, test_preds, fold_scores = trainer.fit_predict(X, y, X_test)
        
        all_oof_preds[full_model_name] = oof_preds
        all_test_preds[full_model_name] = test_preds
        all_scores[full_model_name] = fold_scores 
        
        best_thresh, best_oof = find_best_threshold(y, oof_preds)
        model_best_oof_scores[full_model_name] = best_oof
        
        gc.collect()



--- PROCESSING: XGB on P1 ---
Training XGBClassifier

--- Fold 1 - Accuracy: 0.939690
--- Fold 2 - Accuracy: 0.938013
--- Fold 3 - Accuracy: 0.940526
--- Fold 4 - Accuracy: 0.937699
--- Fold 5 - Accuracy: 0.940072

------ Overall: 0.939200 | Average: 0.939200 ± 0.001133
Tìm thấy Ngưỡng Tốt Nhất: 0.56 | OOF Accuracy: 0.939758

--- PROCESSING: XGB on P2 ---
Training XGBClassifier

--- Fold 1 - Accuracy: 0.941016
--- Fold 2 - Accuracy: 0.938397
--- Fold 3 - Accuracy: 0.940840
--- Fold 4 - Accuracy: 0.937838
--- Fold 5 - Accuracy: 0.940142

------ Overall: 0.939647 | Average: 0.939646 ± 0.001294
Tìm thấy Ngưỡng Tốt Nhất: 0.56 | OOF Accuracy: 0.940058

--- PROCESSING: LGBM on P1 ---
Training LGBMClassifier

--- Fold 1 - Accuracy: 0.940458
--- Fold 2 - Accuracy: 0.939653
--- Fold 3 - Accuracy: 0.941922
--- Fold 4 - Accuracy: 0.939095
--- Fold 5 - Accuracy: 0.940910

------ Overall: 0.940407 | Average: 0.940407 ± 0.000984
Tìm thấy Ngưỡng Tốt Nhất: 0.51 | OOF Accuracy: 0.940421

--- PROCESSIN

In [ ]:
ag_pipeline_list = ['p1','p2']

for pipeline_name in ag_pipeline_list: 
    full_model_name = f"ag_{pipeline_name}" 
    print(f"\n---> Training {full_model_name.upper()} on {pipeline_name}...")
    
    X, y, X_test = get_data(pipeline_type=pipeline_name)
        
    oof_preds, test_preds, fold_scores = train_autogluon_model(
        X, y, X_test, full_model_name, time_limit_sec=1800
    )
    
    all_oof_preds[full_model_name] = oof_preds
    all_test_preds[full_model_name] = test_preds
    all_scores[full_model_name] = fold_scores 
    best_thresh, best_oof = find_best_threshold(y, oof_preds)
    model_best_oof_scores[full_model_name] = best_oof
    gc.collect()


---> Training AG_P1 on p1...
Training AutoGluon model: ag_p1



AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])



------ AutoGluon Overall Score: 0.940561
Tìm thấy Ngưỡng Tốt Nhất: 0.50 | OOF Accuracy: 0.940561

---> Training AG_P2 on p2...
Training AutoGluon model: ag_p2



AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])



------ AutoGluon Overall Score: 0.940715
Tìm thấy Ngưỡng Tốt Nhất: 0.50 | OOF Accuracy: 0.940715


In [ ]:
if not model_best_oof_scores:
    print("[!] Không tìm thấy mô hình nào, bỏ qua huấn luyện pseudo.")
else:
    # --- Chọn model “thầy” tốt nhất dựa trên OOF score ---
    pseudo_model_source = max(model_best_oof_scores, key=model_best_oof_scores.get)
    best_teacher_score = model_best_oof_scores[pseudo_model_source]
    print(f"Tự động chọn '{pseudo_model_source}' làm 'thầy' (OOF Score: {best_teacher_score:.6f}).")
    
    # --- Tạo nhãn giả (soft pseudo-labels) ---
    y_pseudo_labels = pd.Series(all_test_preds[pseudo_model_source], name=CFG.target)
    # Sử dụng OOF preds của model “thầy” để tìm threshold
    oof_preds_teacher = all_oof_preds[pseudo_model_source][:len(y_true)]
    best_thresh, _ = find_best_threshold(y_true, oof_preds_teacher)
    y_pseudo_labels_binary = (y_pseudo_labels > best_thresh).astype(int)
    
    # --- Lấy pipeline gốc của “thầy” để xử lý dữ liệu ---
    source_pipeline_name = pseudo_model_source.split('_')[-1]  # ví dụ 'p1', 'p2', ...
    print(f"Sử dụng xử lý (processing) từ pipeline: {source_pipeline_name}")
    
    _, _, X_pseudo_data = get_data(pipeline_type=source_pipeline_name)
    X_pseudo_data = X_pseudo_data.copy()
    y_pseudo_labels_binary = y_pseudo_labels_binary.copy()
    
    pseudo_data_tuple = (X_pseudo_data, y_pseudo_labels_binary)
    
    # --- Huấn luyện các pipeline pseudo mới (p3 và p4) ---
    pseudo_pipelines = ['p3', 'p4']
    
    for pipeline_name in pseudo_pipelines:
        # Lấy dữ liệu cho pipeline mới, truyền pseudo_data vào
        X, y, X_test = get_data(pipeline_type=pipeline_name, pseudo_data=pseudo_data_tuple)

        full_model_name = f"ag_pseudo_{pipeline_name}" 
        print(f"\n---> Training {full_model_name.upper()} on {pipeline_name}...")

        # --- Huấn luyện AutoGluon ---
        oof_preds, test_preds, fold_scores = train_autogluon_model(
            X, y, X_test, full_model_name, time_limit_sec=1800
        )
        
        # --- Lưu kết quả ---
        all_oof_preds[full_model_name] = oof_preds
        all_test_preds[full_model_name] = test_preds
        all_scores[full_model_name] = fold_scores
        
        # Tìm threshold tốt nhất cho model mới
        best_thresh_p, best_oof_acc_p = find_best_threshold(y, oof_preds[:len(y)])
        print(f"Pipeline {pipeline_name} - Best OOF Accuracy: {best_oof_acc_p:.6f}, Threshold: {best_thresh_p:.4f}")
        
        del X, y, X_test, oof_preds, test_preds
        gc.collect()


Tự động chọn 'ag_p2' làm 'thầy' (OOF Score: 0.940715).
Tìm thấy Ngưỡng Tốt Nhất: 0.50 | OOF Accuracy: 0.940715
Sử dụng xử lý (processing) từ pipeline: p2

---> Training AG_PSEUDO_P3 on p3...
Training AutoGluon model: ag_pseudo_p3



AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
2025-11-18 18:51:25,276	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-11-18 18:51:25,280	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-11-18 18:51:25,284	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.



------ AutoGluon Overall Score: 0.949548
Tìm thấy Ngưỡng Tốt Nhất: 0.50 | OOF Accuracy: 0.949548
Pipeline p3 - Best OOF Accuracy: 0.949548, Threshold: 0.5000

---> Training AG_PSEUDO_P4 on p4...
Training AutoGluon model: ag_pseudo_p4



AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
2025-11-18 19:24:55,145	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-11-18 19:24:55,154	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-11-18 19:24:55,157	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.



------ AutoGluon Overall Score: 0.961937
Tìm thấy Ngưỡng Tốt Nhất: 0.50 | OOF Accuracy: 0.961937
Pipeline p4 - Best OOF Accuracy: 0.961937, Threshold: 0.5000


In [ ]:
os.makedirs('inf', exist_ok=True)

print("Đang lưu các dictionary dự đoán...")

checkpoint_data = {
    'all_oof_preds': all_oof_preds,
    'all_test_preds': all_test_preds,
    'all_scores': all_scores,                
    'model_best_oof_scores': model_best_oof_scores, 
    'y_true': y_true
}

file_path = 'inf/model_predictions_checkpoint.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(checkpoint_data, f)

print(f"Đã lưu thành công vào: {file_path}")

Đang lưu các dictionary dự đoán...
Đã lưu thành công vào: inf/model_predictions_checkpoint.pkl


In [ ]:
all_model_names = list(all_oof_preds.keys())
print(f"Sẽ stack tất cả {len(all_model_names)} model...")

# --- Tạo ma trận OOF ---
for name in ["ag_pseudo_p3", "ag_pseudo_p4"]:
    all_oof_preds[name] = all_oof_preds[name][:len(y_true)]

oof_matrix_all = pd.DataFrame({
    name: all_oof_preds[name] for name in all_model_names
})

# --- Tạo ma trận TEST ---
test_matrix_all = pd.DataFrame({
    name: all_test_preds[name] for name in all_model_names
})

test_matrix_all = test_matrix_all[oof_matrix_all.columns]

# Train meta model
print("Đang huấn luyện Meta-Model (LR + Ridge)...")

# Logistic Regression
meta_model_lr = LogisticRegression(
    random_state=CFG.seed, 
    max_iter=1000
)
oof_lr = cross_val_predict(
    meta_model_lr, oof_matrix_all, y_true, 
    cv=CFG.n_folds, method='predict_proba'
)[:, 1]
meta_model_lr.fit(oof_matrix_all, y_true)
test_lr = meta_model_lr.predict_proba(test_matrix_all)[:, 1]

# Ridge
meta_model_ridge = Ridge(random_state=CFG.seed)
oof_ridge = cross_val_predict(
    meta_model_ridge, oof_matrix_all, y_true, 
    cv=CFG.n_folds, method='predict'
)
meta_model_ridge.fit(oof_matrix_all, y_true)
test_ridge = meta_model_ridge.predict(test_matrix_all)

# Blending 
print("Đang blend các meta-model bằng Optuna...")

oof_meta_matrix = np.vstack([oof_lr, oof_ridge]).T
test_meta_matrix = np.vstack([test_lr, test_ridge]).T
meta_model_names = ['LR_meta', 'Ridge_meta']


def blend_objective(trial):
    w_lr = trial.suggest_float('LR_meta', 0.0, 1.0)
    w_rg = trial.suggest_float('Ridge_meta', 0.0, 1.0)
    w = np.array([w_lr, w_rg])
    w /= w.sum() + 1e-9

    weighted_oof = oof_meta_matrix @ w
    threshold = trial.suggest_float('threshold', 0.4, 0.6, step=0.001)
    preds = (weighted_oof > threshold).astype(int)

    return accuracy_score(y_true, preds)


study_blend = optuna.create_study(direction='maximize')
study_blend.optimize(blend_objective, n_trials=100)

best_weights = np.array([study_blend.best_params['LR_meta'],
                         study_blend.best_params['Ridge_meta']])
best_weights /= best_weights.sum() + 1e-9
best_threshold = study_blend.best_params['threshold']
best_oof_acc = study_blend.best_value

print(f"\nFinal Blend OOF Accuracy: {best_oof_acc:.6f}")
print(f"Final Weights: {dict(zip(meta_model_names, best_weights))}")
print(f"Final Threshold: {best_threshold:.4f}")

oof_final = oof_meta_matrix @ best_weights
test_final = test_meta_matrix @ best_weights

save_submission(
    name="stacker_final",
    test_pred_probs=test_final,
    score=best_oof_acc,
    threshold=best_threshold
)


del oof_matrix_all, test_matrix_all, oof_meta_matrix, test_meta_matrix
gc.collect()


Sẽ stack tất cả 15 model...
Đang huấn luyện Meta-Model (LR + Ridge)...


[I 2025-11-18 19:51:27,188] A new study created in memory with name: no-name-89ca0413-97ff-4a27-a77a-f62650b8fa7b
[I 2025-11-18 19:51:27,205] Trial 0 finished with value: 0.940833193723125 and parameters: {'LR_meta': 0.05306923849543854, 'Ridge_meta': 0.23244370983178142, 'threshold': 0.488}. Best is trial 0 with value: 0.940833193723125.
[I 2025-11-18 19:51:27,222] Trial 1 finished with value: 0.940309655442006 and parameters: {'LR_meta': 0.06966052178428961, 'Ridge_meta': 0.4950159002950887, 'threshold': 0.41200000000000003}. Best is trial 0 with value: 0.940833193723125.
[I 2025-11-18 19:51:27,237] Trial 2 finished with value: 0.9395278382755347 and parameters: {'LR_meta': 0.9377533724892589, 'Ridge_meta': 0.3524973113608485, 'threshold': 0.5640000000000001}. Best is trial 0 with value: 0.940833193723125.
[I 2025-11-18 19:51:27,252] Trial 3 finished with value: 0.9403305969732507 and parameters: {'LR_meta': 0.26473049244400393, 'Ridge_meta': 0.5258812346885587, 'threshold': 0.416000

Đang blend các meta-model bằng Optuna...


[I 2025-11-18 19:51:27,394] Trial 11 finished with value: 0.9398768637962808 and parameters: {'LR_meta': 0.031945479545423, 'Ridge_meta': 0.0016147721103905062, 'threshold': 0.47400000000000003}. Best is trial 0 with value: 0.940833193723125.
[I 2025-11-18 19:51:27,418] Trial 12 finished with value: 0.9404492656503044 and parameters: {'LR_meta': 0.18061302147519243, 'Ridge_meta': 0.15271257281172063, 'threshold': 0.48400000000000004}. Best is trial 0 with value: 0.940833193723125.
[I 2025-11-18 19:51:27,442] Trial 13 finished with value: 0.9406377394315072 and parameters: {'LR_meta': 0.17365952065497275, 'Ridge_meta': 0.16538783049326525, 'threshold': 0.512}. Best is trial 0 with value: 0.940833193723125.
[I 2025-11-18 19:51:27,465] Trial 14 finished with value: 0.9408192327022952 and parameters: {'LR_meta': 0.006448002442549176, 'Ridge_meta': 0.3225266896425348, 'threshold': 0.455}. Best is trial 0 with value: 0.940833193723125.
[I 2025-11-18 19:51:27,486] Trial 15 finished with value


Final Blend OOF Accuracy: 0.941133
Final Weights: {'LR_meta': 0.19510024154508268, 'Ridge_meta': 0.8048997574690531}
Final Threshold: 0.5090
Submission đã lưu tại: submissions/sub_stacker_final_0.941133.csv


2073